In [1]:
import os
import csv
import time
import random
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import re

In [37]:
def open_csv(path):
    with open(path, "r", encoding='utf-8') as f:
        df = pd.read_csv(f)
    return df


def main():
    # path = "C:/Users/Александр/OneDrive/Python/Python projects/023_Autodrom_eda xgboost lgbm catboost/"

    # Open df to scrum
    regions_df = open_csv(f"regions_df.csv")

    for index, row in regions_df.iterrows():
        link = row['link']

        try:
            r = requests.get(link, headers=headers)
            soup = BeautifulSoup(r.text, 'lxml')
            time.sleep(1)
            location = soup.findAll('div', class_='css-inmjwf e162wx9x0')[-1].text.strip().replace('Город: ', '')
            regions_df.at[index, 'location'] = location  # Добавить регион из карточки
        except:
            None
            

    # save to existing file
    regions_df.to_csv(f"{path}regions_df.csv", index=False, encoding='utf-8')


if __name__ == "__main__":
    main()

       region                                               link  \
0  9-yanvarya  https://auto.drom.ru/9-yanvarya/land_rover/fre...   
1      abakan  https://auto.drom.ru/abakan/daihatsu/hijet/893...   
2      abakan  https://auto.drom.ru/abakan/daihatsu/mira/5276...   
3      abakan  https://auto.drom.ru/abakan/daihatsu/mira_e-s/...   
4        aban  https://auto.drom.ru/aban/honda/fit/745681264....   
5        aban  https://auto.drom.ru/aban/mazda/bongo/28602993...   
6        aban  https://auto.drom.ru/aban/mazda/familia/433792...   
7    abatskoe  https://auto.drom.ru/abatskoe/honda/civic/5409...   
8    abatskoe  https://auto.drom.ru/abatskoe/honda/odyssey/49...   
9    abatskoe  https://auto.drom.ru/abatskoe/chevrolet/lanos/...   

                               location  
0  имени 9 Января, Оренбургская область  
1                                   NaN  
2                                Абакан  
3                                Абакан  
4               Абан, Красноярский край  

additional

In [3]:
import os
import csv
import time
import random
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import re

In [6]:
%%time

def open_csv(path):
    with open(path, "r", encoding='utf-8') as f:
        df = pd.read_csv(f)
    return df

def create_csv(path, file_name):
    with open(f"{path}{file_name}.csv", "w", newline="") as f:
        writer = csv.writer(f, delimiter=";")
        row = ['location_en', 'location_ru', 'link']
        writer.writerow(row)
        
# def get_proxies():
#     # You can use any proxy service here, such as 'https://free-proxy-list.net/' or 'https://www.proxyscrape.com/'
#     # Make sure to follow the terms of service of the proxy service you choose
#     proxies = {
#         'http': 'http://<114.129.2.82>:<8081>',
#         'https': 'https://<20.37.207.8>:<8080>'
#     }
#     return proxies


def main():
    # specify paths and create csv, if dont exist
    
    path = "C:/Users/Александр/OneDrive/Python/Python projects/023_Autodrom_eda xgboost lgbm catboost/"
    file_name = "regions_df_add"
    # create_csv(path, file_name)

    # Open df to scrum and create csv to store results:
    regions_df = open_csv(f"{path}{file_name}.csv")
    links_list = regions_df['link'].to_list()
    
    # Session request 
    s = requests.Session()
    s.mount(
        'https://',
        HTTPAdapter(
            max_retries=Retry(
                total=5,
                read=2,
                connect=2,
                backoff_factor=.005,
                allowed_methods=['GET']
            )
        )
    )
    
    data = []
    for i, link in enumerate(links_list):
        if i % 500 == 0 and i != 0:
            print(i, 'ZZzzz')
            time.sleep(600)
            print('[Zag Zag]')
        else:
            ua = UserAgent()
            headers = {
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
                'Cache-Control': 'max-age=0',
                'Connection': 'keep-alive',
                'Sec-Fetch-Dest': 'document',
                'Sec-Fetch-Mode': 'navigate',
                'Sec-Fetch-Site': 'none',
                'Sec-Fetch-User': '?1',
                'Upgrade-Insecure-Requests': '1',
                'User-Agent': ua.random,
                'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
            }
            try:
                r = s.get(link, headers=headers)
                soup = BeautifulSoup(r.text, 'lxml')
                time.sleep(random.uniform(1, 2))
                location_en = link.split('/')[3]
                try:
                    location_ru = soup.findAll('div', class_='css-inmjwf e162wx9x0')[-1].text.strip().replace('Город: ', '')
                except:
                    print(i, '[PASS]', link)
                    location_ru = None

                data.append({
                    'location_en': location_en,
                    'location_ru': location_ru,
                    'link': link
                })
                with open(f"{path}{file_name}.csv", "a", newline="") as f:
                    writer = csv.writer(f, delimiter=";")
                    writer.writerow((location_en, location_ru, link))
            except:
                print(i, '[PASS]', link)
            

if __name__ == "__main__":
    main()

Wall time: 1min 27s
